# 模型:QA架構
訓練模型以區分Good Answer(guitar tab of max rating)與bad Answer(random select) for each songs <br>

Q:Song chord sequence <br>
Good A:Guitar tab chord sequences of max rating  <br>
Bad A:Random select chord sequences from all guitar tab 

# Define function

In [1]:
#pad成158個row(2d matrix,149X3)
def pad_QA(QA):
    data=QA[:]

    for i in range(len(data)):


        a=data[i]
        padding_length=158-len(a)

        data[i]=np.pad(a, [(0, padding_length)], mode='constant')


    return data

In [2]:
from __future__ import print_function

import os

import sys
import random
from time import strftime, gmtime, time

import pickle
import json

import _thread
from scipy.stats import rankdata

random.seed(42)


def log(x):
    print(x)


class Evaluator:
    def __init__(self, conf, model, optimizer=None):
       
        if isinstance(conf, str):
            conf = json.load(open(conf, 'rb'))
        self.model = model(conf)
        #self.path = data_path
        self.conf = conf
        self.params = conf['training']
        optimizer = self.params['optimizer'] if optimizer is None else optimizer
        self.model.compile(optimizer)
       
        self._vocab = None
        self._reverse_vocab = None
        self._eval_sets = None

   

    ##### Loading / saving #####

    def save_epoch(self, epoch,d):
        path='models/'+d
        if not os.path.exists(path):
            os.makedirs(path)
        self.model.save_weights(path+'/weights_epoch_%d.h5' % epoch, overwrite=True)

    def load_epoch(self, epoch,d):
        path='models/'+d
        assert os.path.exists(path+'/weights_epoch_%d.h5' % epoch), 'Weights at epoch %d not found' % epoch
        self.model.load_weights(path+'/weights_epoch_%d.h5' % epoch)

   
   
    ##### Training #####

    def get_time(self):
        return strftime('%Y-%m-%d %H:%M:%S', gmtime())

    def train(self,train_date):
        batch_size = self.params['batch_size']
        nb_epoch = self.params['nb_epoch']
        validation_split = self.params['validation_split']

        #training_set = self.load('train')
        # top_50 = self.load('top_50')

        
        
        
        log('Began training at %s on %d samples' % (self.get_time(), len(questions)))

        
        val_loss = {'loss': 1., 'epoch': 0}

        # def get_bad_samples(indices, top_50):
        #     return [self.answers[random.choice(top_50[i])] for i in indices]

        for i in range(1, nb_epoch+1):
            # sample from all answers to get bad answers
            # if i % 2 == 0:
            #     bad_answers = self.pada(random.sample(self.answers.values(), len(good_answers)))
            # else:
            #     bad_answers = self.pada(get_bad_samples(indices, top_50))
            
            #random.sample第一個參數吃list
            #self.answers.values():dist
            bad_answers = random.sample(list(all_answers), len(good_answers))
            
            bad_answers=np.array(bad_answers)
            print('Fitting epoch %d' % i, file=sys.stderr)
            hist = self.model.fit([questions, good_answers, bad_answers], epochs=1, batch_size=batch_size,
                                  validation_split=validation_split, verbose=1)

            if hist.history['val_loss'][0] < val_loss['loss']:
                val_loss = {'loss': hist.history['val_loss'][0], 'epoch': i}
            print('%s -- Epoch %d ' % (self.get_time(), i) +'Loss = %.4f, Validation Loss = %.4f ' % (hist.history['loss'][0], hist.history['val_loss'][0]) +'(Best: Loss = %.4f, Epoch = %d)' % (val_loss['loss'], val_loss['epoch']))

            self.save_epoch(i,train_date)

        return val_loss

    ##### Evaluation #####

    def prog_bar(self, so_far, total, n_bars=20):
        n_complete = int(so_far * n_bars / total)
        if n_complete >= n_bars - 1:
            print('\r[' + '=' * n_bars + ']', end='', file=sys.stderr)
        else:
            s = '\r[' + '=' * (n_complete - 1) + '>' + '.' * (n_bars - n_complete) + ']'
            print(s, end='', file=sys.stderr)

    

    def get_score(self, verbose=False):
        top1_ls = []
        mrr_ls = []
        c_1, c_2 = 0, 0
        #for each test song
        for i,song_name in enumerate(test_song):
            
            self.prog_bar(i, len(has_2ver[:1000]))
            
            df_tmp=df_test[df_test.song_name==song_name]
            df_tmp=df_tmp.sort_values(by=['rating'], ascending=False)

            #0612:Q的欄位改成songchord_alignment_by_dtw，原song_chord
            q_chord_sequence=df_tmp.iloc[0].song_chord

            questions_row=np.array([])
            for chord in q_chord_sequence:

                index=chord_index.get(chord)
                if index is None:index=np.zeros(1)
                if index is not None:index=np.array(index)
                questions_row=np.hstack([questions_row, index]) if questions_row.size else index


            question=pad_QA([questions_row]*len(df_tmp))


            answers=[]
            for index, row in df_tmp.iterrows():
                answers_row=np.array([])
                for chord in row['chord']:
                    
                    index=chord_index.get(chord)
                    if index is None:index=np.zeros(1)
                    if index is not None:index=np.array(index)
                    answers_row=np.hstack([answers_row, index]) if answers_row.size else index
                answers.append(answers_row)
            answers=pad_QA(answers)


            sims = self.model.predict([question, answers])

            n_good = 1
            max_r = np.argmax(sims)
            max_n = np.argmax(sims[:n_good])

            r = rankdata(sims, method='max')

            
            c_1 += 1 if max_r == max_n else 0
            c_2 += 1 / float(r[max_r] - r[max_n] + 1)

        top1 = c_1 / float(1000)
        mrr = c_2 / float(1000)

        
        print('Top-1 Precision: %f' % top1)
        print('MRR: %f' % mrr)
        top1_ls.append(top1)
        mrr_ls.append(mrr)
        
        return top1_ls, mrr_ls




# Load training data
The following three lists: <br>
1.questions  <br>
2.good answers  <br>
3.all_answers 

In [6]:
import numpy as np

conf = {
    'n_words': 23,
    'question_len': 158,
    'answer_len': 158,
    'margin': 0.02,
    'initial_embed_weights': '../../../Chord_embedding_data/0517/Chord_embedding_sum_lookup_100dim',

    'training': {
        'batch_size': 100,
        'nb_epoch': 100,
        'validation_split': 0.1,
    },

    'similarity': {
        'mode': 'exponential',#cosine,polynomial,sigmoid,euclidean,exponential,gesd,aesd
        'gamma': 0.1,
        'c': 1,
        'd': 2,
        'dropout': 0.5,
    }
}

import pickle

with open('../../../QA_data/061202/questions', 'rb') as pickle_load:
    questions = pickle.load(pickle_load)
    questions=np.array(questions)
with open('../../../QA_data/061202/good_answers', 'rb') as pickle_load:
    good_answers = pickle.load(pickle_load)
    good_answers=np.array(good_answers)
with open('../../../QA_data/061202/all_answers', 'rb') as pickle_load:
    all_answers = pickle.load(pickle_load)
    all_answers=np.array(all_answers)
print('ok')



ok


In [7]:
from keras_models import EmbeddingModel, ConvolutionModel, ConvolutionalLSTM,ConvolutionModel_2
evaluator = Evaluator(conf, model=ConvolutionModel, optimizer='adam')

# Model Training:

In [4]:

    




# train the model

#best_loss = evaluator.train()

print('Finished Training !!')

# evaluate mrr for a particular epoch
# evaluator.load_epoch(best_loss['epoch'])
# top1, mrr = evaluator.get_score(verbose=False)
# print(' - Top-1 Precision:')
# print('   - %.3f on test 1' % top1[0])
# print('   - %.3f on test 2' % top1[1])
# print('   - %.3f on dev' % top1[2])
# print(' - MRR:')
# print('   - %.3f on test 1' % mrr[0])
# print('   - %.3f on test 2' % mrr[1])
# print('   - %.3f on dev' % mrr[2])


ok
Began training at 2019-05-11 11:52:08 on 9023 samples


Fitting epoch 1


Train on 8120 samples, validate on 903 samples
Epoch 1/1
8120/8120 [==============================] - 1021s 126ms/step - loss: 0.0046 - val_loss: 0.0036
2019-05-11 12:09:10 -- Epoch 1 Loss = 0.0046, Validation Loss = 0.0036 (Best: Loss = 0.0036, Epoch = 1)
Train on 8120 samples, validate on 903 samples
Epoch 1/1


Fitting epoch 2


8120/8120 [==============================] - 952s 117ms/step - loss: 0.0033 - val_loss: 0.0024
2019-05-11 12:25:02 -- Epoch 2 Loss = 0.0033, Validation Loss = 0.0024 (Best: Loss = 0.0024, Epoch = 2)
Train on 8120 samples, validate on 903 samples
Epoch 1/1


...

8120/8120 [==============================] - 920s 113ms/step - loss: 0.0027 - val_loss: 0.0018
2019-05-12 13:31:46 -- Epoch 98 Loss = 0.0027, Validation Loss = 0.0018 (Best: Loss = 0.0014, Epoch = 88)
Train on 8120 samples, validate on 903 samples
Epoch 1/1


Fitting epoch 99


8120/8120 [==============================] - 928s 114ms/step - loss: 0.0027 - val_loss: 0.0018
2019-05-12 13:47:14 -- Epoch 99 Loss = 0.0027, Validation Loss = 0.0018 (Best: Loss = 0.0014, Epoch = 88)
Train on 8120 samples, validate on 903 samples
Epoch 1/1


Fitting epoch 100


8120/8120 [==============================] - 933s 115ms/step - loss: 0.0028 - val_loss: 0.0014
2019-05-12 14:02:47 -- Epoch 100 Loss = 0.0028, Validation Loss = 0.0014 (Best: Loss = 0.0014, Epoch = 88)
Finished Training !!


In [5]:
best_loss

{'epoch': 88, 'loss': 0.0013945342991954894}

# 0516以後的 Training

In [8]:
train_date='0614/04'
best_loss = evaluator.train(train_date)

print('Finished Training !!')

Began training at 2019-06-17 05:49:50 on 2510 samples


Fitting epoch 1


Train on 2259 samples, validate on 251 samples
Epoch 1/1
2259/2259 [==============================] - 280s 124ms/step - loss: 0.0068 - val_loss: 0.0072
2019-06-17 05:54:31 -- Epoch 1 Loss = 0.0068, Validation Loss = 0.0072 (Best: Loss = 0.0072, Epoch = 1)
Train on 2259 samples, validate on 251 samples
Epoch 1/1


Fitting epoch 2


2259/2259 [==============================] - 279s 124ms/step - loss: 0.0023 - val_loss: 0.0051
2019-06-17 05:59:10 -- Epoch 2 Loss = 0.0023, Validation Loss = 0.0051 (Best: Loss = 0.0051, Epoch = 2)
Train on 2259 samples, validate on 251 samples
Epoch 1/1


...

Fitting epoch 81


2259/2259 [==============================] - 273s 121ms/step - loss: 0.0013 - val_loss: 0.0014
2019-06-17 11:56:18 -- Epoch 81 Loss = 0.0013, Validation Loss = 0.0014 (Best: Loss = 0.0008, Epoch = 62)
Train on 2259 samples, validate on 251 samples
Epoch 1/1


Fitting epoch 82


1300/2259 [================>.............] - ETA: 1:53 - loss: 0.0013

KeyboardInterrupt: 

# Fit Testing data with best training epoch

讀入cleaning過的df

In [20]:
import pandas as pd
#df = pd.read_pickle('../../../../../data/model_df_cleaned0609')
df = pd.read_pickle('../../../../../data/model_dfdtw_cleaned0612')

In [10]:
chord_vector = {
               
        'C:maj':  [0,4,7],
        'Db:maj': [1,5,8],
        'Db:min': [1,4,8],
        'D:maj':  [2,7,9],
        'Eb:maj': [3,7,10],
        'Eb:min': [3,6,10],
        'E:maj':  [4,8,11],
        'E:min':  [4,7,11],
        'F:maj':  [0,5,9],
        'F:min':  [0,4,9],
        'Gb:maj': [1,6,10],
        'Gb:min': [1,6,9],
        'G:maj':  [2,7,11],
        'G:min':  [2,7,10],
        'Ab:maj': [0,3,8],
        'Ab:min': [3,8,11],
        'A:maj':  [1,4,9],
        'A:min':  [0,4,9],
        'Bb:maj': [2,5,10],
        'Bb:min': [1,5,10],
        'B:maj':  [2,6,11],
        'B:min':  [1,6,11]    
   
    }
chord_index={
    'C:maj':  1,
    'Db:maj': 2,
    'Db:min': 3,
    'D:maj':  4,
    'Eb:maj': 5,
    'Eb:min': 6,
    'E:maj':  7,
    'E:min':  8,
    'F:maj':  9,
    'F:min':  10,
    'Gb:maj': 11,
    'Gb:min': 12,
    'G:maj':  13,
    'G:min':  14,
    'Ab:maj': 15,
    'Ab:min': 16,
    'A:maj':  17,
    'A:min':  18,
    'Bb:maj': 19,
    'Bb:min': 20,
    'B:maj':  21,
    'B:min':  22    

}

In [11]:
#擁有兩張譜以上的歌
#3517筆
has_2ver=df.groupby('song_name').size().loc[lambda x: x>1].keys()


In [9]:
# random.seed(613)
# trainsong_len=len(has_2ver)-1000 #有兩張以上的譜的歌數量減去1000首=訓練歌曲數量
# rd_song=random.sample(list(has_2ver),trainsong_len)
# test_song=has_2ver[~has_2ver.isin(rd_song)]
# df_test=df[df.song_name.isin(test_song)]
# df_test

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1                       _id  \
45            129           129             164  5cada01a9fc5af34a460afcd   
46            130           130             165  5cada01a9fc5af34a460afcf   
47            131           131             166  5cada01a9fc5af34a460afd1   
52            138           138             173  5cada01b9fc5af34a460afe9   
53            139           139             174  5cada01b9fc5af34a460afeb   
66            158           158             193  5cada01f9fc5af34a460b021   
67            159           159             194  5cada01f9fc5af34a460b023   
68            160           160             195  5cada01f9fc5af34a460b025   
69            161           161             196  5cada01f9fc5af34a460b027   
75            168           168             203  5cada01f9fc5af34a460b037   
76            169           169             204  5cada01f9fc5af34a460b039   
91            192           192             227  5cada0239fc5af34a460b06f   
92            193           193             228  5cada0239fc5af34a460b071   
93            194           194             229  5cada0239fc5af34a460b073   
94            195           195             230  5cada0239fc5af34a460b075   
95            196           196             231  5cada0239fc5af34a460b077   
114           219           219             255  5cada0279fc5af34a460b0cb   
115           220           220             256  5cada0279fc5af34a460b0cd   
123           229           229             265  5cada0289fc5af34a460b0ed   
124           230           230             266  5cada0299fc5af34a460b0ef   
130           236           236             272  5cada0299fc5af34a460b0ff   
131           237           237             273  5cada0299fc5af34a460b101   
132           238           238             274  5cada0299fc5af34a460b103   
134           240           240             276  5cada02a9fc5af34a460b10d   
135           241           241             277  5cada02a9fc5af34a460b10f   
136           242           242             278  5cada02a9fc5af34a460b111   
137           243           243             279  5cada02a9fc5af34a460b113   
179           288           288             327  5cada0329fc5af34a460b1b1   
180           289           289             328  5cada0329fc5af34a460b1b3   
181           290           290             329  5cada0329fc5af34a460b1b5   
...           ...           ...             ...                       ...   
17042       19869         19869           21601  5cadaf709fc5af34a461a5c1   
17043       19871         19871           21603  5cadaf709fc5af34a461a5c5   
17044       19872         19872           21604  5cadaf709fc5af34a461a5c7   
17050       19878         19878           21610  5cadaf739fc5af34a461a5db   
17052       19880         19880           21612  5cadaf739fc5af34a461a5df   
17095       19932         19932           21671  5cadaf819fc5af34a461a688   
17096       19933         19933           21672  5cadaf819fc5af34a461a68a   
17097       19934         19934           21673  5cadaf839fc5af34a461a68c   
17098       19935         19935           21674  5cadaf839fc5af34a461a690   
17099       19936         19936           21675  5cadaf839fc5af34a461a692   
17131       19975         19975           21717  5cadaf909fc5af34a461a72e   
17132       19976         19976           21718  5cadaf909fc5af34a461a732   
17133       19977         19977           21719  5cadaf909fc5af34a461a734   
17134       19978         19978           21720  5cadaf909fc5af34a461a738   
17138       19982         19982           21725  5cadaf929fc5af34a461a748   
17139       19983         19983           21726  5cadaf929fc5af34a461a74a   
17140       19984         19984           21727  5cadaf929fc5af34a461a74c   
17141       19985         19985           21728  5cadaf929fc5af34a461a74e   
17156       20003         20003           21746  5cadaf969fc5af34a461a777   
17157       20004         20004           21747  5cadaf969fc5af34a461a779   
171

In [21]:
#拿前1000首有兩張譜以上的歌出來當test songs
test_song=has_2ver[:1000]
df_test=df[df.song_name.isin(test_song)]

df_test

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1                       _id  \
7              76            76             110  5cada0129fc5af34a460af31   
8              77            77             111  5cada0129fc5af34a460af33   
16             93            93             127  5cada0159fc5af34a460af63   
17             94            94             128  5cada0159fc5af34a460af65   
19             97            97             131  5cada0159fc5af34a460af6b   
20             98            98             132  5cada0169fc5af34a460af6d   
22            101           101             135  5cada0169fc5af34a460af79   
23            102           102             136  5cada0169fc5af34a460af7b   
24            103           103             137  5cada0169fc5af34a460af7d   
25            104           104             138  5cada0169fc5af34a460af81   
26            105           105             139  5cada0169fc5af34a460af83   
27            106           106             140  5cada0169fc5af34a460af85   
28            107           107             141  5cada0169fc5af34a460af87   
101           203           203             238  5cada0249fc5af34a460b089   
102           204           204             239  5cada0249fc5af34a460b08d   
103           205           205             240  5cada0249fc5af34a460b08f   
104           206           206             241  5cada0249fc5af34a460b091   
113           218           218             254  5cada0269fc5af34a460b0c7   
114           219           219             255  5cada0279fc5af34a460b0cb   
115           220           220             256  5cada0279fc5af34a460b0cd   
121           227           227             263  5cada0289fc5af34a460b0e5   
122           228           228             264  5cada0289fc5af34a460b0e9   
123           229           229             265  5cada0289fc5af34a460b0ed   
124           230           230             266  5cada0299fc5af34a460b0ef   
125           231           231             267  5cada0299fc5af34a460b0f1   
126           232           232             268  5cada0299fc5af34a460b0f3   
127           233           233             269  5cada0299fc5af34a460b0f5   
128           234           234             270  5cada0299fc5af34a460b0f7   
191           301           301             343  5cada0349fc5af34a460b1d9   
192           302           302             344  5cada0349fc5af34a460b1db   
...           ...           ...             ...                       ...   
16819       19628         19628           21319  5cadaf289fc5af34a461a275   
16820       19629         19629           21320  5cadaf289fc5af34a461a277   
16930       19756         19756           21486  5cadaf519fc5af34a461a455   
16931       19757         19757           21487  5cadaf519fc5af34a461a457   
16933       19759         19759           21489  5cadaf529fc5af34a461a465   
16934       19760         19760           21490  5cadaf529fc5af34a461a467   
16945       19772         19772           21502  5cadaf579fc5af34a461a49b   
16946       19773         19773           21503  5cadaf589fc5af34a461a49d   
16947       19774         19774           21504  5cadaf589fc5af34a461a49f   
16948       19775         19775           21505  5cadaf589fc5af34a461a4a1   
16949       19776         19776           21506  5cadaf589fc5af34a461a4a3   
17046       19874         19874           21606  5cadaf729fc5af34a461a5cd   
17047       19875         19875           21607  5cadaf729fc5af34a461a5cf   
17057       19885         19885           21617  5cadaf759fc5af34a461a5f5   
17058       19886         19886           21618  5cadaf759fc5af34a461a5f9   
17126       19969         19969           21711  5cadaf8f9fc5af34a461a722   
17127       19970         19970           21712  5cadaf8f9fc5af34a461a724   
17153       20000         20000           21743  5cadaf969fc5af34a461a76f   
17154       20001         20001           21744  5cadaf969fc5af34a461a771   
17155       20002         20002           21745  5cadaf969fc5af34a461a773   
171

In [13]:
def status(x) : 
    return pd.Series([x.count(),x.min(),x.idxmin(),x.quantile(.05),x.quantile(.25),x.median(),
                      x.quantile(.75),x.quantile(.95),x.mean(),x.max(),x.idxmax(),x.mad(),x.var(),
                      x.std(),x.skew(),x.kurt()],index=['總數','最小值','最小值位置','5%分位數','25%分位數',
                    '中位數','75%分位數','95%分位數','均值','最大值','最大值位數','平均絕對偏差','方差','標準差','偏度','峰度'])


In [16]:
print(status(df_test['chord_alignment_by_dtw'].str.len()))

總數        3027.000000
最小值         62.000000
最小值位置     1638.000000
5%分位數       72.000000
25%分位數      90.000000
中位數        107.000000
75%分位數     124.000000
95%分位數     145.000000
均值         107.070036
最大值        153.000000
最大值位數     4118.000000
平均絕對偏差      18.400751
方差         486.470308
標準差         22.056072
偏度           0.048291
峰度          -0.836055
dtype: float64


In [17]:
#choose trained model and load weights:
#evaluator = Evaluator(conf, model=ConvolutionModel, optimizer='adam')
evaluator = Evaluator(conf, model=ConvolutionModel, optimizer='adam')


In [22]:
#train_date='0614/02'
#evaluator.load_epoch(best_loss['epoch'])

evaluator.load_epoch(62,train_date)

In [23]:
top1, mrr = evaluator.get_score(verbose=False)

[====================]]

Top-1 Precision: 0.413000
MRR: 0.658988


[====================]